# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load dataframe and display
output_data_file = pd.read_csv('/Users/rollycostillas/Desktop/upenn/homework/myrepository/python-api-challenge/cities.csv')
output_data_file

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Butaritari,86,KI,1651773889,68,3.0707,172.7902,81.68,18.70
1,1,Puerto Ayora,97,EC,1651773890,87,-0.7393,-90.3518,76.96,7.25
2,2,Rikitea,38,PF,1651773890,85,-23.1203,-134.9692,78.24,14.63
3,3,Maceió,40,BR,1651773746,65,-9.6658,-35.7353,85.44,11.50
4,4,New Norfolk,100,AU,1651773891,85,-42.7826,147.0587,56.70,5.99
...,...,...,...,...,...,...,...,...,...,...
538,538,Malakal,63,SS,1651774157,38,9.5334,31.6605,90.70,1.50
539,539,Hirara,100,JP,1651774157,86,24.8000,125.2833,78.10,20.94
540,540,Hopelchén,100,MX,1651774158,33,19.7667,-89.8500,98.11,6.85
541,541,Mayna,49,RU,1651774158,63,54.1135,47.6231,34.93,6.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations
locations = output_data_file[["Lat", "Lng"]]
humidity = output_data_file["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# create ideal new dataframe containing only cities with max temp between 70F & 80F, 
# cloudiness is 0%, and Wind Speed is less than 10mph
vacation_data = output_data_file.loc[(output_data_file["Max Temp"]>70) & 
                                 (output_data_file["Max Temp"]<80) & 
                                 (output_data_file["Cloudiness"]==0) &
                                 (output_data_file["Wind Speed"]<10),:]

# display dataframe
vacation_data

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
75,75,Sakakah,0,SA,1651773921,23,29.9697,40.2064,74.57,5.99
78,78,Keetmanshoop,0,NaN,1651773923,23,-26.5833,18.1333,72.70,6.89
166,166,East London,0,ZA,1651773959,62,-33.0153,27.9116,71.62,8.32
223,223,Bethel,0,US,1651774015,39,41.3712,-73.4140,71.64,8.05
285,285,Saint George,0,US,1651773941,20,37.1041,-113.5841,77.63,6.91
305,305,Baghdad,0,IQ,1651773835,34,33.3406,44.4009,78.71,3.44
312,312,Lawrenceburg,0,US,1651774059,65,38.0373,-84.8966,72.68,5.75
318,318,Herat Province,0,AF,1651774062,38,34.5000,62.0000,71.28,8.43
356,356,Farah,0,AF,1651774078,17,32.5000,62.5000,77.56,4.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = vacation_data.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

# drop any rows where hotel is still null (not found)
hotel_df = hotel_df.dropna(how="any")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBA1fTiE8ZyN48csk_E-u6z2PJczoCoSqpnCzZcBjwjfH2wugopexPCIkYR81e3xenAiHuToTUsjmmpQlendtibNM4hw9m2xZDqCm9DehBPHw2_C9u9U9IxdVjuatUly8O0ZR7ypGoD69lo1ByOXpEqCtpNMDYUz-nZ9pXp5y3GOESb_RZkrFLZm68hCrA7HvOkQEqALoNb0HusiwWWgrp1NuSr0iIoS861hY7tKE7i7CrmfOntS9M_7yghXVDtfdGlO_46P7SOlESXUwiEWMvDJGcl6sWH8OHHkoDQFp3EaRr0zWRB8UDShfxPS5uIquSTU5a2ZyHp9VFmqKxFGRO1zbLprpo7LyM_Sluo0tmjuTsfIyoLbGwtZTXDviRQ5LNpvqRaifOiTPBcuV-z8Vovzo5AbMzbeWgWdjBg7cCIHFpOjwIAV1uEUNVr_yyq",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 29.9452608,
                    "lng": 40.1811211
                },
                "viewport": {
                    "northeast": {
                        "lat": 29.94657357989272,
                        "lng": 40.18234842989271
                    },
                    "southwest": {
                        "lat": 29.943873920

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -26.5818852,
                    "lng": 18.13412
                },
                "viewport": {
                    "northeast": {
                        "lat": -26.58057172010727,
                        "lng": 18.13548327989272
                    },
                    "southwest": {
                        "lat": -26.58327137989272,
                        "lng": 18.13278362010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Quiver Inn Guesthouse",
            "photos": [
                {
                    "height": 3

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBZzdICFv7pQt0KjxTHYlsDAH8K9XHCMnDJYQy0X1EtZjfeHQ9MgfnaMPOBawx4-mWmiG_cu2mRyHOdgl8V7HAp_4mXGjhxd8gbUYHYbZ8gHC1Xo34c4RXr2QRWh1ZdMbWfOT6q0g1H-D_BAxI_TIVbrPDCimpRKxjHVl8HeXRifOpNEcalgyxail8zTgNBoV5n1jHzK02LC5UDPOEfglLnoH5g1j3NCe-cnbX815rVmaqaIcg4nh3-3ogcPkDVaTwHD2cIK528b54QscUD1_FZkLHlIoL1AGzNu5BghkofrbA4zHrYuNQZDHj8byIpBGjPmSN_SPktulERl005JWLfTwM3S-sJsPHYMRq74tQr7tLMr6e7u8p_eJeBTWB11cjT6P4KcXRe4llzyviQlVcMisAuvCkSrq_kKG1XagwmUTDfl_nS5fCqcUxihG1c",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -32.9834877,
                    "lng": 27.9510882
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.98171257010728,
                        "lng": 27.95266547989272
                    },
                    "southwest": {
                        "lat": -32.984412

{
    "html_attributions": [],
    "next_page_token": "Aap_uEA8tFmTt_8jy027LqIig4bECHGT0_dj_e2z3RnjU95-FQD9YUOAFcsTmP3kigCUYAe1cIdfNlscJsH6SaTWUPsbt-ItgMGnbnRkcSOENzlGDWlv1EFuGVJTXq5hSULl-W7ANnGXxuq1hi5tksXjSjhn55PrLJ3VTyOyznwFXaeBCNRrK1XozIT3lqfvpanvHfdF_ryglB5duIxs_mMvHnAU-44eZ54lRvnv0yspV_A7ri7MD69ojPlMzSSOCn5ukwsvErlVjIhGQmkE6XDHxd0pK4U3nGYsEXlNJSDwhOjUwWCLlXmH439_pDXcxuho_b2TKwuq5r8AGeA6jFjD7gzgq2xSCIzra0kiynkFXz7dk1sCd4nGCR5_4ZzUZDmaAIdwKjVcAK2V4vTCQJGRPY6GqY7tVI41DTIg0ziOIrbA-gAYYs4kQot1SaPt4vO0",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 41.3202235,
                    "lng": -73.9968293
                },
                "viewport": {
                    "northeast": {
                        "lat": 41.32157597989271,
                        "lng": -73.99549362010727
                    },
                    "southwest": {
                        "lat": 41.3188763

{
    "html_attributions": [],
    "next_page_token": "Aap_uED6FYYEbfqawHU7i2DudFco1AJh2F1a-Px1qwCv5t3loISsRXLxXYvX5nuWANRM5aGDjtv_ErOkpo3jkoWa4P5T1qeqR2jBkg-CFGeScvqteiKzzHhuKDf0Re4sVM3ybN4gtTRzqdf7AeeNO500xcnBP88LapQRrVkEIpvPZb-2p6ond_5B8hOlrfXVRu7bXphyEUz5FcpvD8P-XCSBOp_j84a1iOFQu5jA3d7BN4ECY3kfm7McDn1kZdMfQQNju5Fxql0lu1EzfdFWit8lboDF48C8ca0h6ut-M4cM1p1BHgUfc70fXT11IlauahCGE4g1K9c4fTSi_aqI5KMaA0I8ssiJUGm6U8I6LZAhbXDm5W6uoHCVGhMVOl3tuRsorYzJIISWT8IYOt76MA_PxA0KaCSjwYytaJgbk-2LUrMpNcM4EFJnfxiVP-zUIPfT",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.2118357,
                    "lng": -113.2702527
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.21289187989272,
                        "lng": -113.2687988701073
                    },
                    "southwest": {
                        "lat": 37.210192

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAAxkSEirf1MXPdxYJBbnZr3htZ6QiLD4wc4INTEOjAcwKkeA14Df62BgNDIM1_K97PQ3XnwKH7G5njE4bc88icYsuTR-GXjes8njytROOff4WEcYxFVnyykvbFtbWgumtd1RZ6Nik_rrGTTc7gz0wuvhpeShXsTQgsYUcF_jqMjKvoWNi9zR3JO446MM9EpprRua75Tkpjw5wyzXXe5sX8oXViJQKm-GE2bZqL13UtOtJTn1v1hbVidfQrhn9wEiZ1wDOA3087HhSuolOYsf5at5F5ituk98Zq19M7nNQpbvwdLjuhKNrcUuuJ2qaZsSu1sRoT_XMy5CdYsa7gknV5ZPCwPFuS4He8n5_ooGiLHfyS1KCKcnLSa0PgmOemm0Lv__ZyUAXqWS7pCbe4jt3rwoG5p6uNGEflNab_vo4BJL94r4ioR1ST-vS-m3Ro",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 33.2912242,
                    "lng": 44.3904889
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.29235297989273,
                        "lng": 44.39198562989272
                    },
                    "southwest": {
                        "lat": 33.289653320

{
    "html_attributions": [],
    "next_page_token": "Aap_uEByryFh_3rJ9_snqgoJIQBj-_7M1ZPRTQK2X7J0sfut6xkfcFE1q1i9t6iMic3EkjOwYz3FdUGToDWqKDtcMJ4OzQgC7TRFx8AZL81tn2pIJazlg8U1cB7eZVjdhDm4RcBbSlSGyDj4JXVWgCBhIqm1Ad6u8e-ZV68nPFBvZgRymBhWnHmzD1CGIGgdAQq_75DVffETQDMFettmlRWcGUWNSmS2s7Eqwe7n0hfDGs5c8YfkuY8z2ZBSSnUy_5jhgBP6fiBtCWMN5WE0dpz-TLtw01psJnY4L_m_uJlYL-yYiHvb0KwX7hE_r7WRkQGLHwkoz1JyxE4ftk3f2CJ_WiO9t1HMWvxoXr_sX9n3HCKeVD2LeldyrW-_yHae33jzgj4NmPq_XoXbWzG_TFsJvbDSxPeJ3uOg6JRo7N_zFoqpkhGIEM0HYUBu9KVoEQ-W",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 38.397533,
                    "lng": -85.37379
                },
                "viewport": {
                    "northeast": {
                        "lat": 38.39875432989272,
                        "lng": -85.37246707010728
                    },
                    "southwest": {
                        "lat": 38.3960546701

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 34.3648643,
                    "lng": 62.2129173
                },
                "viewport": {
                    "northeast": {
                        "lat": 34.36613237989272,
                        "lng": 62.21441707989272
                    },
                    "southwest": {
                        "lat": 34.36343272010728,
                        "lng": 62.21171742010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Arg Hotel",
            "opening_hours": {
                "open_now": true
            },
       

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,75,Sakakah,0,SA,1651773921,23,29.9697,40.2064,74.57,5.99,Raoum Inn Hotel
2,166,East London,0,ZA,1651773959,62,-33.0153,27.9116,71.62,8.32,Blue Lagoon Hotel
3,223,Bethel,0,US,1651774015,39,41.3712,-73.4140,71.64,8.05,Bear Mountain Inn - Overlook Lodge
4,285,Saint George,0,US,1651773941,20,37.1041,-113.5841,77.63,6.91,Best Western Plus Zion West Hotel
5,305,Baghdad,0,IQ,1651773835,34,33.3406,44.4009,78.71,3.44,Babylon Rotana
6,312,Lawrenceburg,0,US,1651774059,65,38.0373,-84.8966,72.68,5.75,Comfort Inn & Suites
7,318,Herat Province,0,AF,1651774062,38,34.5000,62.0000,71.28,8.43,Arg Hotel
8,356,Farah,0,AF,1651774078,17,32.5000,62.5000,77.56,4.59,هوتل خرما


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))